# Create a scrambled FT1 file
The idea is to make a file with mixed data, I think only directions, from several monthly files.

In [50]:
%matplotlib inline
import pyfits
os.chdir(os.path.expandvars('$FERMI/skymodels/P301_monthly'))

In [51]:
class MonthFT1(object):
    def __init__(self, month, 
                 folder='/nfs/farm/g/glast/g/catalog/P8_P301/Source/'):
        self.filename=folder + 'P301_Source_%03d_zmax100.fits'% month 
        self.fits = pyfits.open(self.filename)
        self.events=events=self.fits[1].data
        self.ra = events.RA
        self.dec= events.DEC
        

In [52]:
months = map(MonthFT1, range(1,9))

In [53]:
[len(months[i].ra) for i in range(8)]

[2235172, 2424541, 2525297, 2443990, 2453960, 2427541, 2398897, 2072451]

In [54]:
n = len(months[0].ra)/8; n

279396

In [55]:
ra = months[0].ra.copy()
dec=months[0].dec.copy()
ra[:20]

array([  14.03963947,  205.88999939,  102.68312073,  152.03244019,
         32.52329254,   61.29040146,  121.25073242,   93.77165222,
         94.17869568,  204.07440186,   96.66295624,  354.2097168 ,
         55.74280167,  118.74401855,   67.6678009 ,   44.58832169,
         23.49770164,  199.00662231,   47.19706345,   64.00797272], dtype=float32)

In [56]:
for i in range(1,7):
    ra[i:8*n:8] = months[i].ra[:n]
    dec[i:8*n:8] = months[i].dec[:n]

In [57]:
months[0].ra = ra
months[0].dec=dec

In [58]:
t = months[0].fits
t.writeto(open('scrambled/ft1.fits', 'w'))

In [95]:
check = pyfits.open('scrambled/ft1.fits', mode='update')
check.info()

Filename: scrambled/ft1.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      32   ()              
1    EVENTS      BinTableHDU    206   2235172R x 23C   [E, E, E, E, E, E, E, E, E, D, J, J, I, 3I, 32X, 32X, I, D, E, E, E, E, E]   
2    GTI         BinTableHDU     48   417R x 2C    [D, D]   


In [96]:
check[1].data.RA = ra
check[1].data.DEC=dec
check.flush()

## Merge the livetime cube files
Get the first 8 such.

In [161]:
import pyfits, glob
N=8
path='/nfs/farm/g/glast/g/catalog/P8_P301/Source/'
files = [path+'ltcube_%03d_zmax100.fits' %m for m in range(1,N+1)]
ff= map(pyfits.open, files)
f0=ff[0]
print 'Info: ', f.info()
d=f[1].data
print 'Columns:', d.columns

Info: Filename: /nfs/farm/g/glast/g/catalog/P8_P301/Source/ltcube_008_zmax100.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      27   ()              
1    EXPOSURE    BinTableHDU     61   49152R x 3C   ['40E', 'E', 'E']   
2    WEIGHTED_EXPOSURE  BinTableHDU     61   49152R x 3C   [40E, E, E]   
3    CTHETABOUNDS  BinTableHDU     33   40R x 2C     [E, E]   
4    GTI         BinTableHDU     46   392R x 2C    [D, D]   
 None
Columns: ColDefs(
    name = 'COSBINS'; format = '40E'; unit = 's'
    name = 'RA'; format = 'E'; unit = 'deg'
    name = 'DEC'; format = 'E'; unit = 'deg'
)


Create the average of all 8 EXPOSURE HDU's

In [162]:
for hdu in (1,2):
    print 'HDU', hdu
    cosbins_array  = np.array([f[hdu].data.COSBINS for f in ff]);print cosbins_array.shape
    average = (cosbins_array.sum(axis=0))/N; print average.shape
    #Replace the EXPOSURE of the first cube and save it
    f0[hdu].data.COSBINS = average

HDU 1
(8, 49152, 40)
(49152, 40)
HDU 2
(8, 49152, 40)
(49152, 40)


In [166]:
f0.writeto('/nfs/farm/g/glast/g/catalog/pointlike/fermi/data/P8_P301/scrambled_ltcube.fits',
          clobber=True)

/afs/slac/g/glast/groups/catalog/python/anaconda/lib/python2.7/site-packages/pyfits/file.py:339: UserWarning: Overwriting existing file '/nfs/farm/g/glast/g/catalog/pointlike/fermi/data/P8_P301/scrambled_ltcube.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)


In [167]:
test = pyfits.open('/nfs/farm/g/glast/g/catalog/pointlike/fermi/data/P8_P301/scrambled_ltcube.fits')
test[1].data.COSBINS, test[2].data.COSBINS

(array([[   175.34576416,    418.08270264,    391.01748657, ...,
          37101.65234375,  31687.90820312,  28018.98828125],
        [   168.89424133,    401.27099609,    394.47259521, ...,
          36748.59765625,  31929.79492188,  27994.92578125],
        [   227.04512024,    392.51486206,    384.82571411, ...,
          36591.0390625 ,  31899.15429688,  28267.73046875],
        ..., 
        [   430.27404785,   1172.32751465,   1824.53918457, ...,
          37910.296875  ,  32049.9609375 ,  28677.86914062],
        [   437.57699585,   1330.6628418 ,   1673.54064941, ...,
          37603.0859375 ,  32005.21875   ,  28648.41210938],
        [   417.33868408,   1372.9017334 ,   1566.08789062, ...,
          37890.2890625 ,  31717.3828125 ,  28487.12304688]], dtype=float32),
 array([[   157.78994751,    370.94763184,    348.9128418 , ...,
          33653.03515625,  28662.15429688,  25309.88867188],
        [   152.24008179,    355.86483765,    352.31323242, ...,
          33332.078125

In [175]:
from skymaps import Band,SkyDir
set([round(Band(4).dir(i).b(),2) for i in range(192)])

{-78.28,
 -66.44,
 -54.34,
 -41.81,
 -30.0,
 -19.47,
 -9.59,
 -0.0,
 9.59,
 19.47,
 30.0,
 41.81,
 54.34,
 66.44,
 78.28}